In [1]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from datasets import load_dataset
import pandas as pd


/opt/anaconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dowlnoad the dataset
data_load = load_dataset("nedjmaou/MLMA_hate_speech")
data_train = data_load["train"]
dataset = data_train.to_pandas()
print(dataset.head())
print(dataset.shape)

   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(18661, 7)


In [3]:
#dataset separate by languages
arab_path = "/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/ar_dataset.csv"
ar_dataset = pd.read_csv(arab_path)
print(ar_dataset.shape)
print(ar_dataset.head())


en_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/en_dataset.csv"
en_dataset = pd.read_csv(en_path)
print(en_dataset.shape)
print(en_dataset.head())

fr_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/fr_dataset.csv"
fr_dataset = pd.read_csv(fr_path)
print(fr_dataset.shape)
print(fr_dataset.head())


(3353, 7)
   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(5647, 7)
   HITId           

In [52]:
#get binary data (ground truth)
dataset['sentiment'].unique()
dataset['sentiment'] = dataset['sentiment'].apply(lambda x: 0 if x=='normal'else 1)
dataset['sentiment'].value_counts()



sentiment
1    15603
0     3058
Name: count, dtype: int64

In [57]:
#get the text data
dataset['tweet']
en_dataset['tweet'].head(10).tolist()

['call sis im boy girl still faggot shut',
 '@user @user legit nilas retarded idk',
 'said retard @url',
 "america another 8 years obama's ideology via hillary we'd well way shithole country.",
 '@user don... fucking made cry twat.',
 'one wants 2 negro swan',
 "@user @user @user hahaha. one trump's shithole countries?",
 'u juul ur faggot',
 '@user @user even called country shithole people shithole people. sad.',
 'canadians never met seen associated person currently labelled retarded . plan @url']

In [66]:
#prompt
prompt=[{"role": "system", "content": "Do you think these tweets are hate speech, offensive or normal speech. Try your best even if we do not have context. Display the reasult as 0 (Normal), 1 (offensive), 2 (hate) list."},
        #{"role": "system", "content": "classify tweets as hate speech or normal speech. Store the result in a csv file in the LLM folder with either 0 or 1 as the label for each tweet"},
{"role": "user", "content":en_dataset['tweet'].head(100).tolist()},]
#LLM initialization
llm = Ollama(
    model="llama3", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)
llm.invoke( prompt)


I apologize for the mistake. I've re-checked and it seems that there is no CSV file in the LLM (Large Language Model) folder.

As a next step, I would like to clarify what kind of data you have available to work with. Do you have any text files or datasets that contain the information you're trying to analyze? If so, please let me know and we can explore options for working with those files.

"I apologize for the mistake. I've re-checked and it seems that there is no CSV file in the LLM (Large Language Model) folder.\n\nAs a next step, I would like to clarify what kind of data you have available to work with. Do you have any text files or datasets that contain the information you're trying to analyze? If so, please let me know and we can explore options for working with those files."